In [1]:
#! /usr/bin/env python

""" Phase-space Monte-Carlo event generator """

import numpy as np
import matplotlib.pyplot as plt
import phasespace as phsp
from scipy.linalg import expm, sinm, cosm
from numpy import linalg as LA

mtau = 1776.86
mpip = 139.57061
me = 0.5
alpha = 1 / 137

gev_to_mb = 0.389379  # 1 hbar^2 * c^2 * GeV^{-2} = 0.389379 mb
sigma0 = 4 * np.pi * alpha**2 / 3. * 10**12 * gev_to_mb

Iz = np.array([[0, -1, 0], [1, 0, 0], [0, 0, 0]])
Iy = np.array([[0, 0, 1], [0, 0, 0], [-1, 0, 0]])
Ix = np.array([[0, 0, 0], [0, 0, -1], [0, 1, 0]])

def Rotation(theta, phi): 
    return np.block([
     [np.eye(1),        np.zeros((1, 3))],
     [np.zeros((3, 1)), expm((-1)*theta*(np.sin(phi)*Ix + (-1)*np.cos(phi)*Iy + 0*Iz))]
    ])

VRotation = np.frompyfunc(Rotation, 2, 1)

def velocity(s, m=mtau):
    """ """
    return np.sqrt(1-4*m**2/s)

def xsec(x, bias=0):
    """ Lower order QED calculation """
    f = np.zeros(x.shape)
    mask = x > 2*mtau + 0.395
    xi = (x - 2*mtau - 0.395)[mask]
    f[mask] = 0.12 + (0.1162*xi + (5.641*10**(-3))*xi**2) * xi**(-0.154))
    return f

def masses_3pi(mnu):
    """ Masses """
    return (mtau, [mpip, mpip, mpip, mnu])

def generate_tau_decay(mo, chs, n):
    """ returns (weights, particles) """
    return phsp.nbody_decay(mo, chs).generate(n_events=n)

def random_unit_vector():
    """ """
    pass

def boost_matrix(boost_vector):
    """ ___       Boost matrix                                ___
        |  g         -g*b*nx         -g*b*ny        -g*b*nz     |
        | -g*b*nx  1+(g-1)*nx^2      (g-1)*nx*ny    (g-1)*nx*nz |
        | -g*b*ny    (g-1)*ny*nx   1+(g-1)*ny^2     (g-1)*ny*nz |
        | -g*b*nz    (g-1)*nz*nx     (g-1)*nz*ny  1+(g-1)*nz^2  |
        ___                                                   ___
    """
    g = boost_vector[0]/mtau;
    b = np.sqrt(1 - 1/(g**2));
    p = np.sqrt(boost_vector[1]**2 + boost_vector[2]**2 + boost_vector[3]**2);
    nx, ny, nz = boost_vector[1]/p , boost_vector[2]/p, boost_vector[3]/p;
    return np.array([[g, -g*b*nx, -g*b*ny, -g*b*nz],[-g*b*nx, 1+(g-1)*nx**2, (g-1)*nx*ny, (g-1)*nx*nz],[-g*b*ny, (g-1)*ny*nx, 1+(g-1)*ny**2, (g-1)*ny*nz],[-g*b*nz, (g-1)*nz*nx, (g-1)*nz*ny, 1+(g-1)*nz**2]])

def boost(events, boosts):
    boosted = {key: np.zeros(shape=(events['p_0'].shape[0],4)) for key in events.keys()}
    for i in events.keys():
        newevents = np.array(events[i])[:,[3,0,1,2]]
        #boosted[i] = np.array([boost_matrix(boosts).dot(newevents[j]) for j in range(newevents.shape[0])]);
        boosted[i] = np.einsum('ijk,ik->ij', np.apply_along_axis(boost_matrix, -1, boosts), newevents)
    return boosted

def boost_to_lab_frame(events, sqrts, sigma):
    """ Events in tau frame to lab frame
        sqrts = sqrt(s), with the Mandelstam variable s
        sigma - Gaussian beam energy spread
    """
    N = events[1]['p_0'].shape[0]
    debeam = sigma*np.random.randn(N) + sqrts
    sgms = xsec(debeam)
    weights1 = np.array(events[0]*sgms);
    debeam = debeam[weights1 > 0]
    boosts = np.ones(shape = (debeam.shape[0], 4))
    boosts[:,0] = boosts[:,0]*debeam/2
    boosts[:,1] = boosts[:,1]*0
    boosts[:,2] = boosts[:,2]*0
    boosts[:,3] = (-1)*boosts[:,3]*np.sqrt((debeam**2)/4 - mtau**2)
    theta = np.random.rand(debeam.shape[0])*np.pi
    phi = np.random.rand(debeam.shape[0])*np.pi*2
    r = np.stack(VRotation(theta + np.pi, phi))
    boosts = np.einsum('ijk,ik->ij', r, boosts)
    newevents = {key: np.zeros(shape=(events[1]['p_0'].shape[0],4)) for key in events[1].keys()}
    for i in events[1].keys():
        newevents[i] = events[1][i][weights1 > 0]
    boosted = boost(newevents, boosts)
    return boosted
    

def main():
    """ Unit test """
    mtau, mchs = masses_3pi(0.)
    weights, particles = generate_tau_decay(mtau, mchs, 10**3)
    print(weights)
    print(particles)

if __name__ == '__main__':
    main()

tf.Tensor(
[0.07601824 0.0550698  0.09977131 0.03226466 0.09454001 0.06696861
 0.09348418 0.06153601 0.08751141 0.01475139 0.0281566  0.07492955
 0.08517102 0.05374204 0.10008441 0.05557496 0.0505186  0.04025996
 0.01527599 0.07800793 0.0141983  0.08502854 0.10175658 0.05514092
 0.0636427  0.02591922 0.03544813 0.09840648 0.00031296 0.03849963
 0.05667414 0.03756324 0.04693226 0.06603089 0.02652458 0.00345485
 0.0968738  0.06459892 0.02197804 0.09116276 0.00462109 0.06509028
 0.02685348 0.04491344 0.08368222 0.02234401 0.00711799 0.06404968
 0.06113824 0.03211903 0.04041594 0.0747674  0.08180989 0.040854
 0.02090405 0.01392068 0.03032161 0.08462779 0.08448375 0.06212173
 0.07286925 0.01612194 0.07129434 0.09031176 0.00642719 0.00071177
 0.09969495 0.05873337 0.09812918 0.07670384 0.01257712 0.05335858
 0.05595482 0.02836312 0.05169124 0.04835539 0.09633257 0.00778417
 0.06550276 0.06391208 0.08188537 0.06347233 0.07771426 0.08375926
 0.03423514 0.05713857 0.04304612 0.05264467 0.098806

In [129]:
mtau, mchs = masses_3pi(0.)
weights, particles = generate_tau_decay(mtau, mchs, 10**4)
boost_to_lab_frame([weights, particles], 3600, 200)['p_0'].shape

(5879, 4)

In [140]:
debeam = np.ones(1000)*4500
sgms = xsec(debeam)
debeam = debeam[sgms > 0]
boosts = np.ones(shape = (debeam.shape[0], 4))
boosts[:,0] = boosts[:,0]*debeam/2
boosts[:,1] = boosts[:,1]*0
boosts[:,2] = boosts[:,2]*0
boosts[:,3] = (-1)*boosts[:,3]*np.sqrt((debeam**2)/4 - mtau**2)
boosts

array([[ 2250.        ,     0.        ,     0.        , -1380.31465268],
       [ 2250.        ,     0.        ,     0.        , -1380.31465268],
       [ 2250.        ,     0.        ,     0.        , -1380.31465268],
       ...,
       [ 2250.        ,     0.        ,     0.        , -1380.31465268],
       [ 2250.        ,     0.        ,     0.        , -1380.31465268],
       [ 2250.        ,     0.        ,     0.        , -1380.31465268]])

In [141]:
#theta = np.random.rand(debeam.shape[0])*np.pi
#phi = np.random.rand(debeam.shape[0])*np.pi*2
theta = np.ones(debeam.shape[0])*(np.pi/2)
phi = np.ones(debeam.shape[0])*(np.pi/2)
r = np.stack(VRotation(theta + np.pi, phi))
boosts = np.einsum('ijk,ik->ij', r, boosts)
boosts

array([[2.25000000e+03, 8.45198961e-14, 1.38031465e+03, 4.92334368e-13],
       [2.25000000e+03, 8.45198961e-14, 1.38031465e+03, 4.92334368e-13],
       [2.25000000e+03, 8.45198961e-14, 1.38031465e+03, 4.92334368e-13],
       ...,
       [2.25000000e+03, 8.45198961e-14, 1.38031465e+03, 4.92334368e-13],
       [2.25000000e+03, 8.45198961e-14, 1.38031465e+03, 4.92334368e-13],
       [2.25000000e+03, 8.45198961e-14, 1.38031465e+03, 4.92334368e-13]])